In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
from IPython.display import display
import fileupload
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from cStringIO import StringIO
from ipywidgets import widgets
import time
import os
import io
import pymongo
from pymongo import MongoClient

uploader = fileupload.FileUploadWidget()

ruta = "/home/miguel/Escritorio/Jupyter/NOTICIAS EMAKUNDE"
ficheros = []
nombres = []

def lookForPdf(path):
    for filename in os.listdir(path):
        if filename.endswith(".pdf"):
            ficheros.append(path + '/' + filename)
            nombres.append(filename.replace(".pdf",""))
            #print filename
        elif "." not in filename:
            lookForPdf(path+'/'+filename)



lookForPdf(ruta)
textos = [None]*len(ficheros)
autores =[None]*len(ficheros)
fechas = [None]*len(ficheros)
                
for registro in ficheros:
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    pdfLocal = registro
    fp = file(pdfLocal, 'rb')
    parser = PDFParser(fp)
    doc = PDFDocument(parser)
    #print str(doc.info).decode().encode('utf-8')
    fechaAux = doc.info[0].get('CreationDate', None)
    fechas[ficheros.index(registro)] = fechaAux[2:6]+'-'+fechaAux[6:8]+'-'+fechaAux[8:10]+' '+fechaAux[10:12]+':'+fechaAux[12:14]+':'+fechaAux[14:16]
    autores[ficheros.index(registro)] = doc.info[0].get('Author', None)
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()
    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    text = retstr.getvalue()
    textos[ficheros.index(registro)] = text
    fp.close()
    device.close()
    retstr.close()


cliente = MongoClient()
db = cliente.test_database
noticias = db.noticias
for texto in textos:
    indice = textos.index(texto)
    autor = autores[indice][2:len( autores[indice])]
    nombre = nombres[indice]
    fecha = fechas[indice]
    fichero = ficheros[indice]
    noticia = { 'author': autor, 'title':nombre,
               'link':fichero, 'publishDate':fecha, 'text':texto, 'source':'pdf', 'comentarios':''}
    noticia_id = noticias.insert_one(noticia)

In [4]:
import pandas as pd
import numpy as np
import qgrid
import re
from IPython.display import display
from IPython.display import HTML
from ipywidgets import widgets

style_margin_title = dict (margin = '10px 10px 0px 25%')
style_margin_label = dict (margin = '20px 10px 0px 30%')
style_margin_button = dict (margin = '20px 10px 10px 33%')
style_margin_text = dict (margin = '20px 10px 1px 20%')
style_margin_label_result = dict (margin = '20px 10px 0px 37%',font_size='')

qgrid.nbinstall(overwrite=True)
qgrid.set_defaults(remote_js=True, precision=2)

layoutContainer = widgets.Layout(display='flex',align_items='center',
                                 width='50%', flex_direction='column', justify_content='space-between', margin='0px 0px 0px 25%')
titulo = widgets.HTML(value="<label style=\"font-size:larger\"><strong>Buscador de noticias de pdf<strong></label>", **style_margin_title)
textBox = widgets.Text(**style_margin_text)
button = widgets.Button(description="Buscar", **style_margin_button)
label = widgets.Label(value='Introduzca la palabra clave:',**style_margin_label)
html = widgets.HTML(value="<script type=text/javascript>"
                    + "var labels = document.getElementsByTagName('label');"
     + "for(var i = 0; i < labels.length; i++){"
     + "labels[i].style.color=\"red\";}</script>")
container = widgets.VBox(children=[titulo, label,textBox,button,html], layout=layoutContainer)
container.layout.border = '2px grey solid'



def handle_submit(sender):
    pat = re.compile(textBox.value, re.I)
    pat2 = re.compile('pdf', re.I)
    filas = []
    cursor = db.noticias.find({"text":{'$regex':pat}, "source":{'$regex':pat2}},
                              {"author":1,"link":1, "title":1, "text":1, "publishDate":1, "source":1})
    for estructura in cursor:
        contador = 0
        for palabra in estructura['text'].split(' '):
            if(textBox.value in palabra):
                contador += 1
        filas.append({"fecha publicación":estructura["publishDate"],"autor":estructura["author"], "nombre Archivo":estructura["title"],
                      "fichero":estructura["link"], "apariciones":contador, "fuente":estructura["source"]})

    df = pd.DataFrame(filas)
    df.set_index('apariciones', inplace=True)
    
    labelNoticia = widgets.Label(value='Noticias con la palabra clave: '+ textBox.value, **style_margin_label_result)
    display(labelNoticia)
    qgrid.show_grid(df, grid_options={'fullWidthRows':True,'syncColumnCellResize':True,
                                       'forceFitColumns':True,'rowHeight':40,'enableColumnReorder':True,
                                       'enableTextSelectionOnCells':True,'editable':True})
    #display(HTML(df.to_html(escape=True)))

    
button.on_click(handle_submit)
display(container)

#display(label)
#display(textBox)
#display(button)
